In [2]:
import os
import pyodbc
import numpy as np
import pandas as pd

In [3]:
conn = pyodbc.connect('Driver={SQL Server};' #Connections Removed
                      'Server={Server};'
                      'Database={Database};'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

In [83]:
test = cursor.execute("""SELECT pbp.[GameId]
                                      ,pbp.[EventNum]
                                      ,[SequenceNumber]
                                      ,[Pitch]
                                      , pt.*
                                      , maxseq
                                FROM [Retrosheet].[stage].[PitchByPitch] pbp
                                join stage.PitchType pt
                                    on pbp.pitch = pt.PitchCode
                                join (select Max(sequencenumber) maxseq, Gameid, eventnum 
                                        from stage.PitchByPitch 
                                        group by gameid, eventnum) maxs
                                    on pbp.GameId = maxs.GameId
                                    and pbp.EventNum = maxs.EventNum
                                where left(pbp.GameId,3) = 'ANA'
                                order by pbp.GameId, EventNum""").fetchall()
# add comment

test[0][0][0:3]

'ANA'

In [4]:
import csv

In [96]:
teams = cursor.execute('Select Distinct teamidRetro from Team where yearid >= 2010').fetchall()

i = 1

for t in teams:
    #print(t[0])    
    seq = cursor.execute('''SELECT pbp.[GameId]
                                      ,pbp.[EventNum]
                                      ,[SequenceNumber]
                                      ,[Pitch]
                                      , pt.*
                                      , maxseq
                                FROM [Retrosheet].[stage].[PitchByPitch] pbp
                                join stage.PitchType pt
                                    on pbp.pitch = pt.PitchCode
                                join (select Max(sequencenumber) maxseq, Gameid, eventnum from stage.PitchByPitch group by gameid, eventnum) maxs
                                    on pbp.GameId = maxs.GameId
                                    and pbp.EventNum = maxs.EventNum
                                where left(pbp.GameId,3) = \'{}\'
                                order by pbp.GameId, EventNum
                            '''.format(t[0])).fetchall()
    
    with open('C:\\backups\\TeamPitches\\{}_2010to2018.csv'.format(seq[0][0][0:3]), 'w', newline = '') as writeFile:
        writer = csv.writer(writeFile)    
    
        for row in seq:
            gameid = row[0]
            event = row[1]
            action = row[2]
            pitchtype = row[3]
            isPitch = row[6]
            isStrike = row[7]
            isBall = row[8]
            foul = row[13]
            foulTip = row[14]
            maxseq = row[20]
            
            if action == 1:
                strikes = 0
                balls = 0
            
            currentBalls = balls
            currentStrikes = strikes
            
            if isBall == 1:
                balls += 1
            
            if isStrike == 1:
                strikes += 1
            
            if foul == 1 and strikes > 2:
                strikes -= 1
            
            if pitchtype == 'L' and currentStrikes == 2:
                strikes += 1
            
            if foulTip == 1 and currentStrikes == 2 and pitchnum == maxseq:
                strikes += 1
            
            lines = [gameid, event, action, pitchtype, currentBalls, currentStrikes, None, None, None, None
                    , balls, strikes]
            
            
            writer.writerow(lines)
        
    writeFile.close()
    print('Processed file number {}'.format(i))
    i+=1
        #print('The count before the pitch was {}-{}'.format(currentBalls,currentStrikes))
        #print('The count after the pitch is {}-{}'.format(balls,strikes))


Processed file number 1
Processed file number 2
Processed file number 3
Processed file number 4
Processed file number 5
Processed file number 6
Processed file number 7
Processed file number 8
Processed file number 9
Processed file number 10
Processed file number 11
Processed file number 12
Processed file number 13
Processed file number 14
Processed file number 15
Processed file number 16
Processed file number 17
Processed file number 18
Processed file number 19
Processed file number 20
Processed file number 21
Processed file number 22
Processed file number 23
Processed file number 24
Processed file number 25
Processed file number 26
Processed file number 27
Processed file number 28
Processed file number 29
Processed file number 30
Processed file number 31


In [97]:
os.getcwd()

'C:\\backups\\TeamPitches'

In [91]:
os.chdir('C:\\backups\\TeamPitches')

In [92]:
os.getcwd()

'C:\\backups\\TeamPitches'

In [98]:
for (dirname, dirs, files) in os.walk('.'):
    i = 0
    for filename in files:
        f = os.getcwd() +'\\' + filename
        print(f)
        i += 1
        if i >= 5:
            break

C:\backups\TeamPitches\ANA_2010to2018.csv
C:\backups\TeamPitches\ARI_2010to2018.csv
C:\backups\TeamPitches\ATL_2010to2018.csv
C:\backups\TeamPitches\BAL_2010to2018.csv
C:\backups\TeamPitches\BOS_2010to2018.csv


In [99]:
#Main Script

for (dirname, dirs, files) in os.walk('.'):
    i = 1
    oldcount = int(cursor.execute('Select Count(*) From work.ActionByCount').fetchval())
    for filename in files:
        f = os.getcwd() +'\\' + filename
        sqlcmd = '''Bulk insert work.ActionByCount
                    From ''' + '\'' + f + '\'' + \
                    '''WITH
                    (
                    Firstrow = 1,
                    FieldTerminator = ',',
                    RowTerminator = '\n',
                    Format = 'CSV'
                    )'''
        print('Processing ' + f)
        #print(sqlcmd)
        cursor.execute(sqlcmd)
        cursor.commit()
        newcount = int(cursor.execute('Select Count(*) From work.ActionByCount').fetchval())
        diff = newcount-oldcount
        logcmd = "Insert Into stage.FileImports (ImportedFile, RowsInserted, InsertTime, DestTable) \nValues('{a}', {b}, GetDate(), 'work.ActionByCount')".format(a = f, b = diff)
        #+ f + '\', ' + str(diff) + ', GetDate())'
        print(logcmd)
        cursor.execute(logcmd)
        cursor.commit()
        print(str(i) +' done with file: ' + f)
        print('Rows inserted into work.ActionByCount ' + str(diff))
        oldcount += diff
        i +=1

Processing C:\backups\TeamPitches\ANA_2010to2018.csv
Insert Into stage.FileImports (ImportedFile, RowsInserted, InsertTime, DestTable) 
Values('C:\backups\TeamPitches\ANA_2010to2018.csv', 234269, GetDate(), 'work.ActionByCount')
1 done with file: C:\backups\TeamPitches\ANA_2010to2018.csv
Rows inserted into work.ActionByCount 234269
Processing C:\backups\TeamPitches\ARI_2010to2018.csv
Insert Into stage.FileImports (ImportedFile, RowsInserted, InsertTime, DestTable) 
Values('C:\backups\TeamPitches\ARI_2010to2018.csv', 240563, GetDate(), 'work.ActionByCount')
2 done with file: C:\backups\TeamPitches\ARI_2010to2018.csv
Rows inserted into work.ActionByCount 240563
Processing C:\backups\TeamPitches\ATL_2010to2018.csv
Insert Into stage.FileImports (ImportedFile, RowsInserted, InsertTime, DestTable) 
Values('C:\backups\TeamPitches\ATL_2010to2018.csv', 237268, GetDate(), 'work.ActionByCount')
3 done with file: C:\backups\TeamPitches\ATL_2010to2018.csv
Rows inserted into work.ActionByCount 23726

Insert Into stage.FileImports (ImportedFile, RowsInserted, InsertTime, DestTable) 
Values('C:\backups\TeamPitches\SFN_2010to2018.csv', 232807, GetDate(), 'work.ActionByCount')
26 done with file: C:\backups\TeamPitches\SFN_2010to2018.csv
Rows inserted into work.ActionByCount 232807
Processing C:\backups\TeamPitches\SLN_2010to2018.csv
Insert Into stage.FileImports (ImportedFile, RowsInserted, InsertTime, DestTable) 
Values('C:\backups\TeamPitches\SLN_2010to2018.csv', 233957, GetDate(), 'work.ActionByCount')
27 done with file: C:\backups\TeamPitches\SLN_2010to2018.csv
Rows inserted into work.ActionByCount 233957
Processing C:\backups\TeamPitches\TBA_2010to2018.csv
Insert Into stage.FileImports (ImportedFile, RowsInserted, InsertTime, DestTable) 
Values('C:\backups\TeamPitches\TBA_2010to2018.csv', 237692, GetDate(), 'work.ActionByCount')
28 done with file: C:\backups\TeamPitches\TBA_2010to2018.csv
Rows inserted into work.ActionByCount 237692
Processing C:\backups\TeamPitches\TEX_2010to2018

In [ ]:

#gameids = []
#events = []
#sequences = []
#codes = []
#preballs = []
#prestrikes = []
#ias = []
#iwp = []
#ib = []
#icp = []
#postballs = []
#poststrikes = []
#    
#    #currentTeam = t[0]
#    for g, e, s in seq[['GameId', 'EventNum', 'SequenceNumber']].groupby(['GameId', 'EventNum']).count().reset_index().itertuples(index = False):
#        pitchnum = 1
#        strikes = 0
#        balls = 0
#        for pitch in range(s):        
#            
#            #Set current strikes and balls for before pitch
#            currentStrikes = strikes
#            currentBalls = balls
#            #print(': balls = ' + str(currentBalls) + ' strikes = ' + str(currentStrikes))
#            
#            #Get the values of the series to determine counts
#            processPitch = seq[(seq.GameId == g) & (seq.EventNum == e) & (seq.SequenceNumber == pitchnum)]
#            
#            #print(processPitch.values[0])
#            pitchtype = processPitch.values[0][3]
#            isPitch = processPitch.values[0][6]
#            isStrike = processPitch.values[0][7]
#            isBall = processPitch.values[0][8]
#            #swung = processPitch.values[0][9]
#            #took = processPitch.values[0][10]
#            #buntAttempt = processPitch.values[0][11]
#            #inPlay = processPitch.values[0][12]
#            foul = processPitch.values[0][13]
#            foulTip = processPitch.values[0][14]
#            #pickoff = processPitch.values[0][15]
#            #pickoffCatch = processPitch.values[0][16]
#            #pickoffBase = processPitch.values[0][17]
#            #pitchout = processPitch.values[0][18]
#            #unknown = processPitch.values[0][19]
#               
#            if isBall == 1:
#                balls += 1
#            
#            if isStrike == 1:
#                strikes += 1
#            
#            if foul == 1 and strikes > 2:
#                strikes -= 1
#            
#            if pitchtype == 'L' and strikes == 2:
#                strikes += 1
#            
#            if foulTip == 1 and strikes == 2 and pitchnum == s:
#                strikes += 1
#            
#            gameids.append(g)
#            events.append(e)
#            sequences.append(pitchnum) 
#            codes.append(pitchtype)
#            preballs.append(currentBalls)
#            prestrikes.append(currentStrikes)
#            ias.append(None)
#            iwp.append(None)
#            ib.append(None)
#            icp.append(None)
#            postballs.append(balls)
#            poststrikes.append(strikes)
#            
##            load = load.append({'GameId': g
##                                , 'EventNum': e
##                                , 'SequenceNumber': pitchnum
##                                , 'PitchCode': pitchtype
##                                , 'PrePitchBalls': currentStrikes
##                                , 'PrePitchStrikes': currentBalls
##                                , 'IsAttemptedSteal': None
##                                , 'IsWildPitch': None
##                                , 'IsBlocked': None
##                                , 'IsCatcherPickoff': None
##                                , 'PostPitchBalls': balls
##                                , 'PostPitchStrikes': strikes}, ignore_index = True)
#            
#            #if pitchtype == 'T':
#                #print(pitchtype + ': ' + str(strikes))
#            
#            #print(pitchtype + ': balls = ' + str(balls) + ' strikes = ' + str(strikes))
#            #print(pitchnum)
#            pitchnum +=1
#    
#    load = pd.DataFrame([gameids, events, sequences, codes, preballs, prestrikes, ias
#                         , iwp, ib, icp, postballs, poststrikes]
#                       , columns = ['GameId', 'EventNum', 'SequenceNumber', 'PitchCode', 'PrePitchBalls'
#                                    , 'PrePitchStrikes', 'IsAttemptedSteal', 'IsWildPitch', 'IsBlocked', 'IsCatcherPickoff'
#                                    , 'PostPitchBalls', 'PostPitchStrikes'])
#    
#    
#    if len(load.index) >= 1:
#        load.to_csv('C:\\backups\\TeamPitches\\{}_2010to2018.csv'.format(t[0]), index = False)
#        print('File saved at C:\\backups\\TeamPitches\\{}_2010to2018.csv'.format(t[0]))
#    #print('Select top 10 * from stage.PitchByPitch where left(GameId,3) = \'{}\''.format(t[0]))

In [ ]:
#for g, e, s in seq[['GameId', 'EventNum', 'SequenceNumber']].groupby(['GameId', 'EventNum']).count().reset_index().itertuples(index = False):
#    pitchnum = 1
#    strikes = 0
#    balls = 0
#    for pitch in range(s):        
#        
#        #Set current strikes and balls for before pitch
#        currentStrikes = strikes
#        currentBalls = balls
#        print(': balls = ' + str(currentBalls) + ' strikes = ' + str(currentStrikes))
#        
#        #Get the values of the series to determine counts
#        processPitch = seq[(seq.GameId == g) & (seq.EventNum == e) & (seq.SequenceNumber == pitchnum)]
#        
#        #print(processPitch.values[0])
#        pitchtype = processPitch.values[0][3]
#        isPitch = processPitch.values[0][6]
#        isStrike = processPitch.values[0][7]
#        isBall = processPitch.values[0][8]
#        swung = processPitch.values[0][9]
#        took = processPitch.values[0][10]
#        buntAttempt = processPitch.values[0][11]
#        inPlay = processPitch.values[0][12]
#        foul = processPitch.values[0][13]
#        foulTip = processPitch.values[0][14]
#        pickoff = processPitch.values[0][15]
#        pickoffCatch = processPitch.values[0][16]
#        pickoffBase = processPitch.values[0][17]
#        pitchout = processPitch.values[0][18]
#        unknown = processPitch.values[0][19]
#           
#        if isBall == 1:
#            balls += 1
#        
#        if isStrike == 1:
#            strikes += 1
#        
#        if foul == 1 and strikes > 2:
#            strikes -= 1
#        
#        if pitchtype == 'L' and strikes == 2:
#            strikes += 1
#        
#        if foulTip == 1 and strikes == 2 and pitchnum == s:
#            strikes += 1
#        
#        
#        #if pitchtype == 'T':
#            #print(pitchtype + ': ' + str(strikes))
#        
#        print(pitchtype + ': balls = ' + str(balls) + ' strikes = ' + str(strikes))
#        #print(pitchnum)
#        pitchnum +=1